In [1]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
import logging

%run token.py

updater = Updater(token=token_id, use_context=True)
dispatcher = updater.dispatcher

In [ ]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.INFO)

In [ ]:
start_text = '''
    Olá! \n
    Sou um bot de notícias e informações sobre a Covid-19!\n
    O serviço de jornalismo do Instituto Oswaldo Cruz fez a triagem do conteúdo aqui.
    Sou um bot repassador não oficial! Recebo deles e repasso para vocês.\n
    Não conhece o Instituto Oswaldo Cruz? \n
    Entre em www.ioc.fiocruz.br para conhecer! \n
    (Bot não-oficial administrado por Tiago Lubiana)\n

    (em desenvolvimento)\n"
'''

In [ ]:
def start(update, context):
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=start_text)
        
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [ ]:
with open('14_05_2020_p1.txt', 'r') as file:
        comunicado = file.read()
        comunicado_parseado = comunicado.split("--")

In [ ]:
def get_header(section):
    raw_header = section.split("\n\n")[0]
    header = raw_header.replace("\n", "")
    header = header.replace("\U0001f97c ", "")
    header = header.replace("\U0001f9ea", "")
    return(header)

def get_section_content(section):
    raw_content = section[1:]
    return(raw_content)

In [ ]:
comunicado_headers = [get_header(section) for section in comunicado_parseado]
comunicado_sections = [get_section_content(section) for section in comunicado_parseado]
comunicado_header_to_section = dict(zip(comunicado_headers, comunicado_sections ))

In [ ]:
comunicado_headers

In [ ]:
updater.start_polling()

In [ ]:
# Repetição de código pois não conheço tão bem o módulo telegram
# O código grita por refatoração. 


def epid(update, context):
    text = comunicado_header_to_section["📊 Epidemiologia"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('epid', epid)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def ajuda(update, context):
    text = '''
    O robô possui atualmente funções para os seguintes temas
    - 📊 Epidemiologia (/epid)
    - 🔬 Pesquisa científica (/pesquisa)
    - Diagnóstico (/diag)
    - 🏡 Distanciamento social (/distanciamento)
    - 🗂️ Política (/politica)
    - Saúde Pública (/saude)
    - 🌐 Internacional (/internacional)
    - 🔍 Recortes da pandemia (/recortes)
    - ✅ Curtas (/curtas)  
    '''
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
ajuda_handler = CommandHandler('ajuda', ajuda)
dispatcher.add_handler(ajuda_handler)

In [ ]:
def diag(update, context):
    text = comunicado_header_to_section[" Diagnóstico"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('diag', diag)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def politica(update, context):
    text = comunicado_header_to_section["🗂️ Política"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('politica', politica)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def internacional(update, context):
    text = comunicado_header_to_section["🌐 Internacional"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('internacional', internacional)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def recortes(update, context):
    text = comunicado_header_to_section['🔍 Recortes da pandemia']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('recortes', recortes)
dispatcher.add_handler(handler_for_this_call)

In [16]:
def curtas(update, context):
    text = comunicado_header_to_section['✅ Curtas']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('curtas', curtas)
dispatcher.add_handler(handler_for_this_call)

In [17]:
def distanciamento(update, context):
    text = comunicado_header_to_section['🏡 Distanciamento social']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('distanciamento', distanciamento)
dispatcher.add_handler(handler_for_this_call)

In [18]:
def pesquisa(update, context):
    text = comunicado_header_to_section['🔬 Pesquisa científica']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('pesquisa', pesquisa)
dispatcher.add_handler(handler_for_this_call)

In [19]:
def saude(update, context):
    text = comunicado_header_to_section['Saúde Pública']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('saude', saude)
dispatcher.add_handler(handler_for_this_call)